In [21]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from src import deepTarget_utils as bio_utils
import re
import src.RNA
from  src.RNA import cofold
from src import my_deepTarget_utils as utils
from src.deepTarget_utils import find_candidate

In [16]:
poss_site_level = "../data/13059_2014_500_MOESM1_ESM=site-level human miRNA-target site pairs from miRecords=.csv"
data = pd.read_csv(poss_site_level)

In [17]:
mirna_fasta_file = "../data/miRNA.fasta"
mrna_fasta_file = "../data/3UTR.fasta"
mirna_ids, mirna_seqs, mrna_ids, mrna_seqs = bio_utils.read_fasta(mirna_fasta_file, mrna_fasta_file)

In [18]:
data["seq"] = data.mRNA_ID.apply(lambda x: mrna_seqs[mrna_ids.index(x)])
data["miseq"] = data.miR_ID.apply(lambda x: mirna_seqs[mirna_ids.index(x)])

In [22]:
for index in range(len(data)):
    datum = data.loc[index]
    mirna = datum.miseq
    target = datum.seq

In [11]:
from multiprocessing import Pool

def par_func(i):
    return cofold(data.loc[i].seq)[0]

p = Pool(7)
forms = p.map(par_func, list(range(len(data))))

In [13]:
from collections import Counter
c = 0
x = 0
for index in range(len(forms)):
    start = data.Start_position.loc[index]
    end = data.End_position.loc[index]
    site = forms[index][start:end]
    
    counts=Counter(site)
    prob_dot_site = counts["."]/len(site)
    counts_all=dict(Counter(forms[index]))
    prob_dot = counts_all["."]/len(forms[index])
    x += prob_dot_site
    if  prob_dot_site > prob_dot:
        c+=1
c/len(forms), x/len(forms)

(0.47928994082840237, 0.38865531258231134)

In [ ]:
# bad results, no correlation